In [1]:
import os
import sys
import math
import operator
import numpy as np
import pandas as pd
import pickle as pkl
import tifffile as tif
from keras.layers import Dense
from keras.layers import Conv2D
from multiprocessing import Pool
from keras.utils import Sequence
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.models import Sequential 
from keras.layers import Reshape
from collections import OrderedDict
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Permute
from keras.models import load_model, save_model
os.chdir("../../")

Using TensorFlow backend.


In [17]:
class RNN_train:
    
    def __init__(self, directory, time_steps):
        
        self.path = directory
        self.time_steps = time_steps
        
        with open("Data/hierarchy_data.pkl","rb") as f:
            hd = pkl.load(f)
        with open("Data/class_encoding.pkl","rb") as f:
            self.classes = pkl.load(f)
        with open("Data/order_encoding.pkl","rb") as f:
            self.orders = pkl.load(f)
        with open("Data/family_encoding.pkl","rb") as f:
            self.families = pkl.load(f)
        with open("Data/genus_encoding.pkl","rb") as f:
            self.genuses = pkl.load(f)
        with open("Data/specie_encoding.pkl","rb") as f:
            self.species = pkl.load(f)
            
        self.train_seq = []
        self.test_seq = []
        
        for cls in hd.keys():
            for order in hd[cls].keys():
                for family in hd[cls][order].keys():
                    for genus in hd[cls][order][family].keys():
                        for specie in hd[cls][order][family][genus]:
                            for im in os.listdir(self.path+"train/"+str(self.classes[cls])+"/"+str(self.orders[order])
                                                 +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)):
                                self.train_seq.extend([self.classes[cls],self.orders[order],
                                                       self.families[family],self.genuses[genus],
                                                       specie])
        for cls in hd.keys():
            for order in hd[cls].keys():
                for family in hd[cls][order].keys():
                    for genus in hd[cls][order][family].keys():
                        for specie in hd[cls][order][family][genus]:
                            for im in os.listdir(self.path+"test/"+str(self.classes[cls])+"/"+str(self.orders[order])
                                                 +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)):
                                self.test_seq.extend([self.classes[cls],self.orders[order],
                                                       self.families[family],self.genuses[genus],
                                                       specie])
        self.train_seq = np.array(self.train_seq)
        self.test_seq = np.array(self.test_seq)
        
        np.random.shuffle(self.train_seq)
        np.random.shuffle(self.test_seq)
        
        self.xtrain = []
        self.xtest= []
        
        for i in range(self.train_seq.shape[0]-self.time_steps):
            self.xtrain.append(self.train_seq[i])
            
        self.ytrain = np.array(self.xtrain[self.time_steps:])
        self.xtrain = np.array(self.xtrain[:-self.time_steps])
        
        for i in range(self.test_seq.shape[0]-self.time_steps):
            self.xtest.append(self.test_seq[i])
        
        self.ytest = np.array(self.xtest[self.time_steps:])
        self.xtest = np.array(self.xtest[:-self.time_steps])
        
        self.xtrain = np.reshape(self.xtrain,(self.xtrain.shape[0],1,1))
        self.ytrain = np.reshape(self.ytrain,(self.ytrain.shape[0],1,1))
        self.xtest = np.reshape(self.xtest,(self.xtest.shape[0],1,1))
        self.ytest = np.reshape(self.ytest,(self.ytest.shape[0],1,1))
        
        print(self.xtrain.shape, self.ytrain.shape, self.xtest.shape, self.ytest.shape)
        
    def RNN_Model(self):
        
        model = Sequential()
        model.add(LSTM(500, return_sequences=True, input_shape=(1,1)))
        model.add(LSTM(400, return_sequences=True))
        model.add(LSTM(300, return_sequences=True))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(1))
        model.summary()
        
        model.compile(loss='mse', optimizer='rmsprop', metrics=['mae'])
        model.fit(self.xtrain, self.ytrain, epochs=500)
        scores = model.evaluate(self.xtest, self.ytest)

In [18]:
ob = RNN_train("Data/Hierarchial Data/", 1)

(764898, 1, 1) (764898, 1, 1) (327813, 1, 1) (327813, 1, 1)


In [19]:
ob.RNN_Model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1, 500)            1004000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 1, 400)            1441600   
_________________________________________________________________
lstm_6 (LSTM)                (None, 1, 300)            841200    
_________________________________________________________________
dense_7 (Dense)              (None, 1, 256)            77056     
_________________________________________________________________
dense_8 (Dense)              (None, 1, 128)            32896     
_________________________________________________________________
dense_9 (Dense)              (None, 1, 1)              129       
Total params: 3,396,881
Trainable params: 3,396,881
Non-trainable params: 0
_________________________________________________________________


KeyboardInterrupt: 